In [1]:
import pandas as pd 
from dotenv import load_dotenv
import json
from openai import OpenAI
import os 

load_dotenv()


def embed_chunk(df, i, client, window):
    window = df.iloc[i : i + window]
    ids = window['id'].values.tolist()

    result = client.embeddings.create(
        input=window['text'].values.tolist(),
        model="text-embedding-3-small"
    )

    return [
        {'id': ids[i], 'embedding': x.embedding} 
        for i, x in enumerate(result.data)
    ]
    

def embed_df(df, client, file_name):
    WINDOW = 1000

    embeddings = []
    i = 0
    while i + WINDOW < len(df):
        embeddings.extend(embed_chunk(df, i, client, WINDOW))
        i += WINDOW

    if i < len(df):
        embeddings.extend(embed_chunk(df, i, client, WINDOW))

    del(df)

    output_name = file_name.split('.', 1)[0] + '.jsonl'

    with open('transformed/' + output_name, 'wt') as file:
        for row in embeddings:
            file.write(json.dumps(row) + '\n')

    del(embeddings)


def embed_file(file_name: str, directory: str):
    TOKEN_LIMIT = 50
    df = pd.read_parquet(os.path.join(directory, file_name))

    df['text'] = df.text.apply(lambda x: ' '.join(x.split()[: TOKEN_LIMIT]))

    client = OpenAI(api_key=os.environ.get("API_KEY"))
    embed_df(df, client, file_name)

In [2]:
i = 0
for file_name in os.listdir('source'):
    print(file_name)
    embed_file(file_name, 'source')
    i += 1
    if i == 2:
        break

train-00000-of-00041.parquet
train-00001-of-00041.parquet
